Importing Libraries

In [10]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.linear_model import LogisticRegressionCV
import xgboost as xgb 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

%matplotlib inline

Importing the dataset

In [12]:
train_values = pd.read_csv('./dataset/train_values.csv')
train_labels = pd.read_csv('./dataset/train_labels.csv')
test_values = pd.read_csv('./dataset/test_values.csv')
submission = pd.read_csv('./dataset/submission_format.csv')

Understanding the dataset

In [15]:
train_values.shape

(260601, 39)

In [22]:
test_values.shape

(86868, 39)

In [23]:
train_values.head().transpose()

,0,1,2,3,4
building_id,802906,28830,94947,590882,201944
geo_level_1_id,6,8,21,22,11
geo_level_2_id,487,900,363,418,131
geo_level_3_id,12198,2812,8973,10694,1488
count_floors_pre_eq,2,2,2,2,3
age,30,10,10,10,30
area_percentage,6,8,5,6,8
height_percentage,5,7,5,5,9
land_surface_condition,t,o,t,t,t
foundation_type,r,r,r,r,r


In [21]:
train_values.dtypes

building_id                                int64
geo_level_1_id                             int64
geo_level_2_id                             int64
geo_level_3_id                             int64
count_floors_pre_eq                        int64
age                                        int64
area_percentage                            int64
height_percentage                          int64
land_surface_condition                    object
foundation_type                           object
roof_type                                 object
ground_floor_type                         object
other_floor_type                          object
position                                  object
plan_configuration                        object
has_superstructure_adobe_mud               int64
has_superstructure_mud_mortar_stone        int64
has_superstructure_stone_flag              int64
has_superstructure_cement_mortar_stone     int64
has_superstructure_mud_mortar_brick        int64
has_superstructure_c

Dataset Preprocessing

Check for missing, NaN values

In [24]:
train_values.isnull().values.any()
train_labels.isnull().values.any()

#The dataset contains no missing values.

False

Check for Outliers

Converting categorical variables into Numeric form

In [28]:
#Columns with categorical data
num_cols = train_values._get_numeric_data().columns
categories = list(set(train_values.columns) - set(num_cols))
categories

['other_floor_type',
 'legal_ownership_status',
 'land_surface_condition',
 'foundation_type',
 'position',
 'plan_configuration',
 'ground_floor_type',
 'roof_type']

In [30]:
for i in categories:
    print(i, set(train_values[i]))

other_floor_type {'s', 'j', 'q', 'x'}
legal_ownership_status {'w', 'r', 'a', 'v'}
land_surface_condition {'o', 'n', 't'}
foundation_type {'h', 'r', 'i', 'w', 'u'}
position {'o', 's', 't', 'j'}
plan_configuration {'n', 'f', 'c', 'm', 's', 'u', 'q', 'd', 'o', 'a'}
ground_floor_type {'f', 'x', 'z', 'm', 'v'}
roof_type {'n', 'q', 'x'}


In [45]:
#One hot Encodings
train_values_new = pd.get_dummies(train_values)
test_values_new = pd.get_dummies(test_values)

print('Training dataset :',train_values_new.shape)
print()
print('Test dataset :',test_values_new.shape)

Training dataset : (260601, 39)

Test dataset : (86868, 69)


In [53]:
#Factors Method

#Preparing Training Data
df = train_values
for col_name in categories:
        df[col_name]= df[col_name].astype('category')
        df[col_name] = df[col_name].cat.codes
print("Train Data Shape: ", df.shape)

#Preparing Test Data
df_test = test_values
for col_name in categories:
        df_test[col_name]= df_test[col_name].astype('category')
        df_test[col_name] = df_test[col_name].cat.codes
print("Test Data Shape: ", df_test.shape)

Train Data Shape:  (260601, 39)
Test Data Shape:  (86868, 39)


Basic Summary Statistic

In [48]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
building_id,260601.0,525675.482773,304544.999032,4.0,261190.0,525757.0,789762.0,1052934.0
geo_level_1_id,260601.0,13.900353,8.033617,0.0,7.0,12.0,21.0,30.0
geo_level_2_id,260601.0,701.074685,412.710734,0.0,350.0,702.0,1050.0,1427.0
geo_level_3_id,260601.0,6257.876148,3646.369645,0.0,3073.0,6270.0,9412.0,12567.0
count_floors_pre_eq,260601.0,2.129723,0.727665,1.0,2.0,2.0,2.0,9.0
age,260601.0,26.535029,73.565937,0.0,10.0,15.0,30.0,995.0
area_percentage,260601.0,8.018051,4.392231,1.0,5.0,7.0,9.0,100.0
height_percentage,260601.0,5.434365,1.918418,2.0,4.0,5.0,6.0,32.0
land_surface_condition,260601.0,1.695427,0.696040,0.0,2.0,2.0,2.0,2.0
foundation_type,260601.0,2.119036,0.579154,0.0,2.0,2.0,2.0,4.0


Exploratory Data Analysis